In [33]:
import pandas as pd
import numpy as np

## FX Overlay
## ----------------------------------------------------------------------------------
## 1. Loading data (GDP, Inflation, Futures, 2y yields)
## 2. Building THREE macro signals per region:
##      - Business Cycle   (GDP YoY – Inflation YoY)
##      - Monetary Policy  (– 12m change in 2y yield)
##      - Risk Sentiment   (12m equity futures return)
#       - Here we are missing the FX/ trade data 

## 3. Combines them into a single macro FX score per region & month

## 4. Standardizes scores cross-sectionally (z-scores across regions each month)

## 5. Maping the z-score into a Discrete Hedge Ratio:
##       strong positive -> 0% hedge (expect FX appreciation)
##       mid positive   -> 50% hedge
##       mid negative   -> 75% hedge
##       strong negative -> 100% hedge (expect FX depreciation)

## 6. Signal at t used for t+1 (Shifting the hedge ratio forward by one month)

## 7. Outputs a long-format table: Date, Region, FXScore_z, HedgeRatio_next_month


# --------------------------------------------------------------------------------------------
# 1. Configuration & Data Loading
# --------------------------------------------------------------------------------------------

# Define your local paths here
gdp_path  = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/GDP_forecasts.csv"      # Expected cols: Date, US, GB, EU, CH, JP, AU, EM
inf_path  = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/Inflation_forecasts.csv" # Expected cols: Date, US, GB, EU, CH, JP, AU, EM
fut_path  = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/Futures.xlsx"
y2_path   = r"C:/Users/Sedláček/Documents/UZH/PMP/Macro_momentum/PMP_December_8/Data/2_years_yields.xlsx"

# Load Data
# Note: For CSVs, we use read_csv. For Excel, read_excel.
# We explicitly parse dates and set the index to 'Date' immediately for easier handling.

gdp_forecasts = pd.read_csv(gdp_path, parse_dates=["Date"], index_col="Date")
inf_forecasts = pd.read_csv(inf_path, parse_dates=["Date"], index_col="Date")
fut           = pd.read_excel(fut_path, parse_dates=["Date"], index_col="Date")
y2            = pd.read_excel(y2_path,  parse_dates=["Date"], index_col="Date")


In [34]:
# --------------------------------------------------------------------------------------------
# 2. Pre-processing (UNCHANGED)
# --------------------------------------------------------------------------------------------
def clean_columns(df):
    new_cols = []
    for col in df.columns:
        clean_name = str(col).split('_')[0]
        new_cols.append(clean_name)
    df.columns = new_cols
    return df

y2 = clean_columns(y2)
fut = clean_columns(fut)
fut = fut.rename(columns={"UK": "GB"})

gdp_monthly = gdp_forecasts.asfreq('MS').ffill().sort_index()
inf_monthly = inf_forecasts.asfreq('MS').ffill().sort_index()
fut_monthly = fut.asfreq('MS').ffill().sort_index()
y2_monthly  = y2.asfreq('MS').ffill().sort_index()

target_regions = ["US", "GB", "EU", "CH", "JP", "AU", "EM"]

gdp_monthly = gdp_monthly.reindex(columns=target_regions)
inf_monthly = inf_monthly.reindex(columns=target_regions)
fut_monthly = fut_monthly.reindex(columns=target_regions)
y2_monthly  = y2_monthly.reindex(columns=target_regions)

valid_regions = target_regions

In [35]:
# --------------------------------------------------------------------------------------------
# 3. Signal Generation (UNCHANGED)
# --------------------------------------------------------------------------------------------
gdp_change = gdp_monthly[valid_regions].diff(12) 
inf_change = inf_monthly[valid_regions].diff(12) 
bc_raw = (gdp_change + inf_change) / 2.0

mp_raw = y2_monthly[valid_regions].diff(12)
rs_raw = fut_monthly[valid_regions].pct_change(12)

C:\Users\Sedláček\AppData\Local\Temp\ipykernel_23468\3246442192.py:9: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rs_raw = fut_monthly[valid_regions].pct_change(12)


In [37]:
# --------------------------------------------------------------------------------------------
# 4. Longitudinal Standardization (UNCHANGED)
# --------------------------------------------------------------------------------------------
def calculate_historic_zscore(df, min_periods=12):
    rolling_mean = df.expanding(min_periods=min_periods).mean()
    rolling_std  = df.expanding(min_periods=min_periods).std()
    z_score = (df - rolling_mean) / rolling_std.replace(0, np.nan)
    return z_score.clip(-3, 3)

bc_z = calculate_historic_zscore(bc_raw)
mp_z = calculate_historic_zscore(mp_raw)
rs_z = calculate_historic_zscore(rs_raw)

# --------------------------------------------------------------------------------------------
# 5. Composite Signal Construction (UNCHANGED)
# --------------------------------------------------------------------------------------------
sum_df = bc_z.fillna(0) + mp_z.fillna(0) + rs_z.fillna(0)
count_df = bc_z.notna().astype(int) + mp_z.notna().astype(int) + rs_z.notna().astype(int)
composite_signal = sum_df / count_df.replace(0, np.nan)

In [38]:
# --------------------------------------------------------------------------------------------
# 6. Cross-Sectional Ranking (UNCHANGED)
# --------------------------------------------------------------------------------------------
def cross_sectional_zscore(df):
    return df.sub(df.mean(axis=1), axis=0).div(df.std(axis=1), axis=0)

final_z_score = cross_sectional_zscore(composite_signal)

# --------------------------------------------------------------------------------------------
# NEW STEP 6.5: Calculate Relative Score vs USD
# --------------------------------------------------------------------------------------------

# 1. Isolate the US Score
us_score = final_z_score['US']

# 2. Subtract US Score from all other regions (Region_Score - US_Score)
# If Result > 0: Region is Stronger than US
# If Result < 0: Region is Weaker than US
relative_fx_score = final_z_score.sub(us_score, axis=0)

# 3. Remove US column (Since US vs US is always 0 and irrelevant for hedging)
relative_fx_score = relative_fx_score.drop(columns=['US'], errors='ignore')

# --------------------------------------------------------------------------------------------
# 7. Hedge Ratio Mapping (UPDATED for Relative Strength)
# --------------------------------------------------------------------------------------------

def get_hedge_ratio(rel_score):
    if pd.isna(rel_score): return np.nan
    
    # Logic:
    # High Positive Score -> Region stronger than US -> Expect FX appreciation -> 0% Hedge
    # High Negative Score -> Region weaker than US -> Expect FX depreciation -> 100% Hedge
    
    if rel_score > 1.0: return 0.0     # Strong Bullish vs USD
    if rel_score > 0.0: return 0.50    # Mild Bullish vs USD
    if rel_score > -1.0: return 0.75   # Mild Bearish vs USD
    return 1.0                         # Strong Bearish vs USD

# Apply logic to the RELATIVE score, not the raw Z score
hedge_ratios = relative_fx_score.applymap(get_hedge_ratio)
hedge_ratios_next_month = hedge_ratios.shift(1)

C:\Users\Sedláček\AppData\Local\Temp\ipykernel_23468\3590349219.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  hedge_ratios = relative_fx_score.applymap(get_hedge_ratio)


In [39]:
# --------------------------------------------------------------------------------------------
# 8. Export (UPDATED)
# --------------------------------------------------------------------------------------------

df_hedge = hedge_ratios_next_month.stack(dropna=False).to_frame(name='Hedge_Ratio_Next_Month')

# Note: We stack the relative_fx_score now, not the raw final_z_score
df_score = relative_fx_score.shift(1).stack(dropna=False).to_frame(name='FX_Score_Z_vs_USD')

output = df_hedge.join(df_score)
output = output.reset_index()
output.columns = ['Date', 'Region', 'Hedge_Ratio_Next_Month', 'FX_Score_Z_vs_USD']

# Clean up empty rows
output = output.dropna(subset=['Hedge_Ratio_Next_Month', 'FX_Score_Z_vs_USD'], how='all')

print("Output Tail (Relative to USD):")
print(output.tail(30))

Output Tail (Relative to USD):
           Date Region  Hedge_Ratio_Next_Month  FX_Score_Z_vs_USD
3984 2025-05-01     GB                    0.50           0.513578
3985 2025-05-01     EU                    0.50           0.333087
3986 2025-05-01     CH                    0.50           0.493301
3987 2025-05-01     JP                    0.00           1.925708
3988 2025-05-01     AU                    1.00          -1.182028
3990 2025-06-01     GB                    0.50           0.589352
3991 2025-06-01     EU                    0.50           0.175136
3992 2025-06-01     CH                    0.50           0.446551
3993 2025-06-01     JP                    0.00           1.840654
3994 2025-06-01     AU                    1.00          -1.254218
3996 2025-07-01     GB                    0.50           0.636275
3997 2025-07-01     EU                    0.50           0.230227
3998 2025-07-01     CH                    0.50           0.589614
3999 2025-07-01     JP                    0.0

C:\Users\Sedláček\AppData\Local\Temp\ipykernel_23468\3264215218.py:5: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_hedge = hedge_ratios_next_month.stack(dropna=False).to_frame(name='Hedge_Ratio_Next_Month')
C:\Users\Sedláček\AppData\Local\Temp\ipykernel_23468\3264215218.py:8: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_score = relative_fx_score.shift(1).stack(dropna=False).to_frame(name='FX_Score_Z_vs_USD')
